
### Import Libraries

In [0]:
pip install msal

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
# Define imports and config dictionary

from pyspark.sql.functions import col, count, array, concat, lit, explode, udf, concat
from pyspark.sql.types import StringType, ArrayType
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

import msal
import requests


### Read and transform Dataverse tables from datalake

In [0]:
powerappsuserrole_response = (spark.read
  .format("csv")
  .option("header",True)
  .load("/mnt/datalake/datalakepowerplatform/bronze/ghq_guardianangel_prod/admin_powerplatformuserrole")
)

In [0]:
powerappsuserrole = powerappsuserrole_response.select('admin_app','admin_powerplatformuser')

In [0]:
powerappsuser_response = (spark.read
  .format("csv")
  .option("header",True)
  .load("/mnt/datalake/datalakepowerplatform/bronze/ghq_guardianangel_prod/admin_powerplatformuser")
)

In [0]:
powerappsuser = powerappsuser_response.select('admin_powerplatformuserid','admin_type')

powerappsuser_user = powerappsuser.filter(powerappsuser.admin_type == 'User')
powerappsuser_group = powerappsuser.filter(powerappsuser.admin_type == 'Group')

powerappsuser_user = powerappsuser_user.select(
    col('admin_powerplatformuserid').alias('group'),
    col('admin_powerplatformuserid').alias('id'),
    lit('powerplatformuser').alias('type')
)

powerappsuser_group = powerappsuser_group.select(
    col('admin_powerplatformuserid').alias('group'),
    concat(lit('https://graph.microsoft.com/v1.0/groups/'),col('group'),lit('/transitiveMembers')).alias('url')
)

In [0]:
security_group_response = (spark.read
  .format("json")
  .load("/mnt/datalake/powerapps/environments/environments.json")
)

In [0]:
security_group_response = security_group_response.filter(security_group_response.securityGroup != "-")

environment_group = security_group_response.select(
    'environmentId',
    'securityGroup'
)

security_group = security_group_response.select(
    col('securityGroup').alias('group'),
    concat(lit('https://graph.microsoft.com/v1.0/groups/'),col('group'),lit('/transitiveMembers')).alias('url')
)

In [0]:
powerappsuserrole_response = (spark.read
  .format("csv")
  .option("header",True)
  .load("/mnt/datalake/datalakepowerplatform/bronze/ghq_guardianangel_prod/admin_powerplatformuserrole")
)

In [0]:
powerappsuserrole = powerappsuserrole_response.select(
    'admin_app',
    col('admin_powerplatformuser').alias('group')
)

In [0]:
admin_app_response = (spark.read
  .format("csv")
  .option("header",True)
  .load("/mnt/datalake/datalakepowerplatform/bronze/ghq_guardianangel_prod/admin_app")
)

In [0]:
admin_app = admin_app_response.select(
    'admin_appid',
    'admin_appenvironmentid',
    'admin_appidstring'
)


### Create function to make graph API call

In [0]:
app_id = dbutils.secrets.get(scope = 'keyvault', key = 'AUGHQITSBIid')
app_secret = dbutils.secrets.get(scope = 'keyvault', key = 'AUGHQITSBIvalue')

config = {
  'client_id': app_id,
  'client_secret': app_secret,
  'authority': 'https://login.microsoftonline.com/cef04b19-7776-4a94-b89b-375c77a8f936',
  'scope': ['https://graph.microsoft.com/.default']
}

In [0]:
# Define a function that takes a 'url' parameter and executes a graph call.
def make_graph_call(url):
    # Crie uma instância MSAL fornecendo os parâmetros client_id, authority e client_credential
    client = msal.ConfidentialClientApplication(
        config['client_id'], authority=config['authority'], client_credential=config['client_secret']
    )

    # Primeiramente, tente buscar um token de acesso no cache
    token_result = client.acquire_token_silent(config['scope'], account=None)

    # Se o token não estiver disponível no cache, adquira um novo do Azure AD
    if not token_result:
        token_result = client.acquire_token_for_client(scopes=config['scope'])

    # Se o token estiver disponível, execute a consulta no Graph
    if 'access_token' in token_result:
        headers = {'Authorization': 'Bearer ' + token_result['access_token']}
        graph_results = []

        while url:
            try:
                graph_result = requests.get(url=url, headers=headers).json()
                # Extraia apenas o atributo 'id' de cada item em 'value'
                graph_results.extend([item['id'] for item in graph_result.get('value', [])])
                # Atualize a URL para a próxima página, se disponível
                url = graph_result.get('@odata.nextLink')
            except Exception as e:
                print(f"Error: {e}")
                break
    else:
        print(token_result.get('error'))
        print(token_result.get('error_description'))
        print(token_result.get('correlation'))

    return graph_results

In [0]:
make_graph_call_udf = udf(make_graph_call, ArrayType(StringType()))


### Apply function to dataframe rows

In [0]:
powerappsuser_group_expanded = powerappsuser_group.withColumn('result', make_graph_call_udf(powerappsuser_group.url))

powerappsuser_group_expanded = powerappsuser_group_expanded.select(
    'group',
    explode('result').alias('id'),
    lit('powerplatformuser').alias('type')
)

powerappsuser_all = powerappsuser_user.union(powerappsuser_group_expanded)

In [0]:
security_group_expanded = security_group.withColumn('result', make_graph_call_udf(security_group.url))

security_group_expanded = security_group_expanded.select(
    'group',
    explode('result').alias('id'),
    lit('security_group').alias('type')
)


### Merge tables

In [0]:
app_secutiry_group = admin_app.join(
    environment_group,
    admin_app.admin_appenvironmentid == environment_group.environmentId
)

app_secutiry_group = app_secutiry_group.select(
    col('admin_appid').alias('admin_app'),
    col('securityGroup').alias('group')
)

app_group = powerappsuserrole.union(app_secutiry_group).distinct()

In [0]:
group_user = powerappsuser_all.union(security_group_expanded).distinct()


### Write data in the datalake

In [0]:
app_group.write.mode('overwrite').parquet('/mnt/datalake/datalakepowerplatform/gold/coe/app_group')

In [0]:
# group_user.write.parquet('/mnt/datalake/datalakepowerplatform/gold/coe/group_user')

group_user.write.mode('overwrite').csv('/mnt/datalake/datalakepowerplatform/gold/coe/group_user' , header=True)

In [0]:
#----------------------------------------------------------------------
#
#
#

In [0]:
# data = [
#     ('group1',
#      '33e52479-9742-47de-a569-fc3e89cbd416'),
#     ('group21',
#      '0c8460b5-46fc-4836-823f-f47709edf415')
# ]

# schema = StructType([
#     StructField("name",StringType(),True),
#     StructField("group",StringType(),True)
# ])

# df = spark.createDataFrame(data=data,schema=schema)


In [0]:
# df = df.select(
#     'name',
#     'group',
#     concat(lit('https://graph.microsoft.com/v1.0/groups/'),col('group'),lit('/transitiveMembers')).alias('url')
# )

In [0]:
# df2 = df.withColumn('result', make_graph_call_udf(df.url))

In [0]:
# df2 = df2.select(
#     'group',
#     explode('result').alias('id')
# )

In [0]:
# df2.write.mode('overwrite').parquet('/mnt/datalake/datalakepowerplatform/gold/coe/df2')

In [0]:
# # Criar uma Spark session
# spark = SparkSession.builder.appName("example").getOrCreate()

# rows = powerappsuser_group.collect()

# for item in rows:
#     results = make_graph_call(item.url)

#     if results:
#         # Criar DataFrame com colunas 'id' e 'group'
#         result_df = spark.createDataFrame([(result, item.group) for result in results], ["id", "group"])

#         # Obter o nome do grupo
#         group = item.group

#         # Gravar o DataFrame no Data Lake
#         result_df.write.mode('overwrite').parquet(f'/mnt/datalake/datalakepowerplatform/gold/coe/df2/{group}_result')


In [0]:
# # Criar uma Spark session
# spark = SparkSession.builder.appName("example").getOrCreate()

# # DataFrame principal para armazenar todos os resultados
# all_results_df = None

# rows = powerappsuser_group.collect()

# for item in rows:
#     results = make_graph_call(item.url)

#     if results:
#         # Criar DataFrame com colunas 'id' e 'group'
#         current_result_df = spark.createDataFrame([(result, item.group) for result in results], ["id", "group"])
        
#         # Se o DataFrame principal não existir, atribuir o atual, senão, fazer a união (append)
#         if all_results_df is None:
#             all_results_df = current_result_df
#         else:
#             all_results_df = all_results_df.union(current_result_df)

